In [2]:
#! pip install cache_pandas
#! pip install soccerdata
#! pip install scikit-learn
#! pip install tpot
#! pip install xgboost
#! pip install optuna
import soccerdata as sd 
import pandas as pd
import os
import numpy as np
from cache_pandas import cache_to_csv
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score
from tpot import TPOTClassifier
import xgboost as xgb
from sklearn.model_selection import cross_val_score

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)



[10/08/23 14:06:39] INFO     No custom team name replacements found. You can configure these in       ]8;id=660863;file:///Users/Gautham/miniconda3/envs/dl_hw0/lib/python3.8/site-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=606524;file:///Users/Gautham/miniconda3/envs/dl_hw0/lib/python3.8/site-packages/soccerdata/_config.py#84\84]8;;\
                             /Users/Gautham/soccerdata/config/teamname_replacements.json.                          

                    INFO     No custom league dict found. You can configure additional leagues in    ]8;id=16954;file:///Users/Gautham/miniconda3/envs/dl_hw0/lib/python3.8/site-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=83354;file:///Users/Gautham/miniconda3/envs/dl_hw0/lib/python3.8/site-packages/soccerdata/_config.py#162\162]8;;\
                             /Users/Gautham/soccerdata/config/league_dict.json.                                    

In [32]:
#! pip install understat
! pip install asyncio
from understat import Understat
import asyncio
import json

import aiohttp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 2.1 MB/s eta 0:00:00


In [484]:
@cache_to_csv("cache/prem.csv", refresh_time=300)
def get_prem_data():
    directory = '/Users/Gautham/Downloads/archive-2/Datasets'
    total_df = pd.DataFrame()

    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        
        if os.path.isfile(f) and filename[-3:] == 'csv':
            print(f)
            df = pd.read_csv(f, encoding = 'unicode_escape')
            total_df = pd.concat([total_df,df], join='outer')
    return total_df

In [99]:
@cache_to_csv("cache/schedule.csv", refresh_time=0)
def get_schedule():
    fbref = sd.FBref('ENG-Premier League', '2223-2324')
    sched23 = fbref.read_schedule()

    sched23 = sched23.droplevel('league')
    sched23 = sched23.droplevel('season')

    sched23 = sched23.rename(columns={'date':'Date', 'home_team':'HomeTeam', 'away_team':'AwayTeam', 'referee': 'Referee'})
    sched23['Date'] = pd.to_datetime(sched23['Date'], yearfirst=True).dt.strftime('%d/%m/%y')
    sched23['FTHG'] = sched23['score'].str[0].astype('float')
    sched23['FTAG'] = sched23['score'].str[-1].astype('float')
    sched23[['BbAv>2.5', 'BbAv<2.5', 'BbAvH', 'BbAvD', 'BbAvA']] = None
    sched23['HomeTeam'] = sched23['HomeTeam'].map(mapping)
    sched23['AwayTeam'] = sched23['AwayTeam'].map(mapping)
    sched23 = sched23.reset_index(drop=True)
    return sched23

In [118]:
@cache_to_csv("cache/current_season.csv", refresh_time=0)
def get_current_season():
    df = pd.DataFrame()
    years =  ['0102', '0203','0304','0405','0506','0607','0708','0809','0910','1011','1112','1213','1314','1415','1516','1617','1718','1819','1920','2021','2122','2223','2323']
    years = ['2223', '2324']

    for year in years: 
        history = sd.MatchHistory('ENG-Premier League', year)#'2000-2023')
        sched = history.read_games()
        df = pd.concat([df, sched])


    sched23 = df
    #history = sd.MatchHistory('ENG-Premier League', '2000')#'2000-2023')
    #sched23 = history.read_games()

    sched23 = sched23.droplevel('league')
    sched23 = sched23.droplevel('season')

    sched23 = sched23.rename(columns={'date':'Date', 'home_team':'HomeTeam', 'away_team':'AwayTeam', 'referee': 'Referee'})
    sched23['Date'] = pd.to_datetime(sched23['Date'], yearfirst=True).dt.strftime('%d/%m/%y')
    
    #sched23[['BbAv>2.5', 'BbAv<2.5', 'BbAvH', 'BbAvD', 'BbAvA']] = None
    sched23['HomeTeam'] = sched23['HomeTeam'].map(mapping)
    sched23['AwayTeam'] = sched23['AwayTeam'].map(mapping)
    sched23 = sched23.reset_index(drop=True)
    return sched23

    

In [983]:
@cache_to_csv("cache/elo.csv", refresh_time=86400)
def get_elo_data(full_df):
    clubelo = sd.ClubElo('ENG-Premier League')  
    # create elo df
    elo_df = pd.DataFrame()
    for club in full_df['Team'].unique():
        print('hi', club)
        try:
            elo = clubelo.read_team_history(club)
            elo_df = pd.concat([elo_df,elo])
        except:
            print(club)

    # add nottingham data
    elo_df = pd.concat([elo_df, clubelo.read_team_history('Forest')])
    elo_df['team'].replace(['Forest'], "Nott'm Forest", regex=True, inplace=True)
    elo_df = elo_df.reset_index()
    # merge elo with full df
    out = full_df.merge(elo_df[['team', 'elo', 'from', 'to']], how='left', left_on=['Team'], right_on=['team']) 
    out = out.query('Date.between(`from`, `to`)')
    out = out.merge(elo_df[['team', 'elo', 'from', 'to']], how='left', left_on=['Opponent'], right_on=['team']) 
    out = out.query('Date.between(`from_y`, `to_y`)')
    out = out.reset_index(drop=True)
    out['ELODif'] = out['elo_x'] - out['elo_y']
    out = out.rename(columns={'elo_x':'TeamELO', 'elo_y':'OpponentELO'})
    out = out.drop(columns=['from_x', 'from_y', 'to_x', 'to_y', 'team_x', 'team_y'])
    return out

In [329]:
def home_away(df):
        if 'Team' not in df.columns:
            df = rename_cols(df)
        if 'GameID' not in df.columns:
            df['GameID'] = range(df.shape[0])

        c1 = df.filter(like='Team').columns
        c2 = c1.str.replace('Team', 'Opponent')
        
        swap = df.rename(columns={**dict(zip(c1, c2)), **dict(zip(c2, c1))})
        swap['Venue'] = 'Away'
        df['Result'] = np.where(df['Result'] == 'H', 'W', np.where(df['Result'] == 'A', 'L', df['Result']))
        swap['Result'] = np.where(swap['Result'] == 'H', 'L', np.where(swap['Result'] == 'A', 'W', swap['Result']))
        #swap['Result'] = np.where(swap['Result'] == 'H', 'A', np.where(swap['Result'] == 'A', 'H', 'D'))
        #print(swap)
        df = pd.concat([df,swap])
        
        #df['Result'] = np.where(df['Result'] != 'W', 'L', df['Result'])
        #df = df.rename(columns={'Team1':'Team', 'Team2':'Opponent'})
        df = df.sort_values(by='GameID').reset_index(drop=True)
        return df

In [849]:
@cache_to_csv("cache/processed.csv", refresh_time=60)
def preprocess(total_df, source):
    total_df['TotalGoals'] = total_df['FTHG'] + total_df['FTAG']
    total_df['Date'] = pd.to_datetime(total_df['Date'], dayfirst=True)
    
    total_df = total_df.sort_values(by='Date')
    total_df['GameID'] = range(len(total_df))

    df = total_df
    #return df
    #return df.filter(like='>2.5')#.isna().sum()
    if source == 'csv':
        df = total_df[['GameID', 'Date', 'HomeTeam', 'AwayTeam', 'TotalGoals', 'FTHG', 'FTAG', 'FTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'BbAv>2.5', 'BbAv<2.5', 'AvgC>2.5', 'AvgC<2.5', 'B365H', 'B365D', 'B365A', 'AvgCH', 'AvgCD', 'AvgCA', 'B365>2.5', 'B365<2.5']]
        
        #df['O2.5'] = np.where(df['AvgC>2.5'].isna(),np.where(~df['BbAv>2.5'].isna(),df['BbAv>2.5'],df['B365>2.5']),df['AvgC>2.5'])
        #df['O2.5'] = df['B365>2.5']
        #df['U2.5'] = df['B365<2.5']
        df['O2.5'] = np.where(df['BbAv>2.5'].isna(),df['B365>2.5'],df['BbAv>2.5'])
        df['U2.5'] = np.where(df['AvgC<2.5'].isna(),np.where(~df['BbAv<2.5'].isna(),df['BbAv<2.5'],df['B365<2.5']),df['AvgC<2.5'])
        #df['U2.5'] = np.where(df['BbAv<2.5'].isna(),df['B365<2.5'],df['BbAv<2.5'])

        #['O2.5'].isna().sum())
        
        df['HomeTeamWinOdds'] = df['B365H']#np.where(df['AvgCH'].isna(),df['BbAvH'],df['AvgCH'])
        df['AwayTeamWinOdds'] = df['B365A']#np.where(df['AvgCA'].isna(),df['BbAvA'],df['AvgCA'])
        df['DrawOdds'] = df['B365D']#np.where(df['AvgCD'].isna(),df['BbAvD'],df['AvgCD'])
        
        df.columns = ['GameID', 'Date', 'HomeTeam', 'AwayTeam', 'TotalGoals', 'HomeTeamGoals', 'AwayTeamGoals', 'Result', 'Referee', 'HomeTeamShots', 'AwayTeamShots', 'HomeTeamShotsonTarget', 'AwayTeamShotsonTarget', 'HomeTeamFouls', 'AwayTeamFouls', 'HomeTeamCorners', 'AwayTeamCorners', 'HomeTeamYellowCards', 'AwayTeamYellowCards', 'HomeTeamRedCards', 'AwayTeamRedCards', 'BbO2.5', 'BbU2.5', 'AvgO2.5', 'AvgU2.5','BbHome', 'BbDraw', 'BbAway', 'AvgH', 'AvgD', 'AvgA', 'B365O', 'B365U', 'O2.5', 'U2.5', 'HomeTeamWinOdds',  'AwayTeamWinOdds', 'DrawOdds']

        df = df[['GameID', 'Date', 'HomeTeam', 'AwayTeam', 'TotalGoals', 'HomeTeamGoals', 'AwayTeamGoals', 'Result', 'Referee', 'HomeTeamShots', 'AwayTeamShots', 'HomeTeamShotsonTarget', 'AwayTeamShotsonTarget', 'HomeTeamFouls', 'AwayTeamFouls', 'HomeTeamCorners', 'AwayTeamCorners', 'HomeTeamYellowCards', 'AwayTeamYellowCards', 'HomeTeamRedCards', 'AwayTeamRedCards', 'O2.5', 'U2.5', 'HomeTeamWinOdds', 'DrawOdds', 'AwayTeamWinOdds']]
        df = df.sort_values(by='Date')

    '''
    c1 = df.filter(like='HomeTeam').columns

    c2 = c1.str.replace('HomeTeam', 'Team')
    df = df.rename(columns={**dict(zip(c1, c2)), **dict(zip(c2, c1))})
    c1 = df.filter(like='AwayTeam').columns

    c2 = c1.str.replace('AwayTeam', 'Opponent')

    df = df.rename(columns={**dict(zip(c1, c2)), **dict(zip(c2, c1))})
    df['Venue'] = 'Home'

    c1 = df.filter(like='Team').columns
    c2 = c1.str.replace('Team', 'Opponent')

    swap = df.rename(columns={**dict(zip(c1, c2)), **dict(zip(c2, c1))})
    swap['Venue'] = 'Away'
    #print(swap)
    df = pd.concat([df,swap])
    df['Team'] = df['Team'].map(mapping)
    df['Opponent'] = df['Opponent'].map(mapping)
    #df = df.rename(columns={'Team1':'Team', 'Team2':'Opponent'})
    df = df.sort_values(by='GameID').reset_index(drop=True)
    '''
    df = home_away(df)
    return df

In [850]:
d = preprocess(new_df2023, 'csv')#.dropna(subset=['O2.5'])
d.dropna(subset=['O2.5'])#.isna().sum()#subset=['BbMx>2.5'])

,GameID,Date,Team,Opponent,TotalGoals,TeamGoals,OpponentGoals,Result,Referee,TeamShots,OpponentShots,TeamShotsonTarget,OpponentShotsonTarget,TeamFouls,OpponentFouls,TeamCorners,OpponentCorners,TeamYellowCards,OpponentYellowCards,TeamRedCards,OpponentRedCards,O2.5,U2.5,TeamWinOdds,DrawOdds,OpponentWinOdds,Venue
1778,889,2002-11-16,Arsenal,Tottenham,3,3,0,H,M Riley,17,6,12,5,15,10,3,1,0,2,0,1,1.72,2.00,1.533,3.50,5.500,Home
1779,889,2002-11-16,Tottenham,Arsenal,3,0,3,A,M Riley,6,17,5,12,10,15,1,3,2,0,1,0,1.72,2.00,5.500,3.50,1.533,Away
1780,890,2002-11-16,Chelsea,Middlesbrough,1,1,0,H,P Dowd,13,7,5,4,9,11,5,7,2,1,0,0,1.80,1.90,1.727,3.20,4.500,Home
1781,890,2002-11-16,Middlesbrough,Chelsea,1,0,1,A,P Dowd,7,13,4,5,11,9,7,5,1,2,0,0,1.80,1.90,4.500,3.20,1.727,Away
1782,891,2002-11-16,Man City,Charlton,1,0,1,A,G Barber,15,13,4,4,10,16,7,5,0,4,0,0,1.83,1.83,1.667,3.40,4.500,Home
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16893,8446,2023-09-17,Arsenal,Everton,1,1,0,H,S Hooper,13,8,4,1,10,12,11,1,1,1,0,0,1.67,2.23,1.500,4.50,6.000,Away
16894,8447,2023-09-17,Bournemouth,Chelsea,0,0,0,D,D Coote,13,14,4,6,15,20,1,7,1,5,0,0,1.67,2.19,4.330,3.75,1.800,Home
16895,8447,2023-09-17,Chelsea,Bournemouth,0,0,0,D,D Coote,14,13,6,4,20,15,7,1,5,1,0,0,1.67,2.19,1.800,3.75,4.330,Away
16896,8448,2023-09-18,Forest,Burnley,2,1,1,D,R Jones,14,10,4,3,16,10,5,4,2,3,0,1,1.92,1.87,2.200,3.50,3.250,Home


In [8]:
def create_predictors(df):
    df['VenueCode'] = df['Venue'].astype('category').cat.codes
    df['OpponentCode'] = df['Opponent'].astype('category').cat.codes
    df['Target'] = (df['TotalGoals'] > 2.5).astype('int')
    return df.reset_index(drop=True)

In [969]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values(by='Date')
    rolling_stats = group[cols].rolling(3, min_periods=2, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

@cache_to_csv("cache/rolling.csv", refresh_time=60)
def create_rolling(merged_df):
    cols = ['TotalGoals','TeamGoals', 'OpponentGoals', 'TeamShots', 'OpponentShots', 'TeamShotsonTarget', 'OpponentShotsonTarget', 'TeamCorners', 'OpponentCorners']
    new_cols = [f'{c}Rolling' for c in cols]
    df_rolling = merged_df.groupby('Team').apply(lambda x: rolling_averages(x, cols, new_cols))
    df_rolling = df_rolling.droplevel('Team')
    df_rolling = df_rolling.sort_values('GameID')
    df_rolling.index = range(df_rolling.shape[0])
    return df_rolling

In [982]:
print('getting prem')
prem_df = get_prem_data()
print('getting schedule')
sched23 = get_current_season()
#sched23 = get_schedule()

new_df2023 = pd.concat([prem_df, sched23])
new_df2023.to_csv('cache/total_df.csv')
print(new_df2023.columns)
print('preprocess')
df1 = preprocess(new_df2023, 'csv')

print('create predictors')
full_df = create_predictors(df1)
print('getting elo')
merged_df = get_elo_data(full_df)
print('create rolling')
df_rolling = create_rolling(merged_df)
print('predicting')


#sched23

getting prem
getting schedule


[09/21/23 19:47:59] INFO     Saving cached data to /Users/Gautham/soccerdata/data/MatchHistory        ]8;id=398382;file:///Users/Gautham/miniconda3/envs/dl_hw0/lib/python3.8/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=101414;file:///Users/Gautham/miniconda3/envs/dl_hw0/lib/python3.8/site-packages/soccerdata/_common.py#89\89]8;;\

INFO:root:Saving cached data to /Users/Gautham/soccerdata/data/MatchHistory


                    INFO     Saving cached data to /Users/Gautham/soccerdata/data/MatchHistory        ]8;id=277370;file:///Users/Gautham/miniconda3/envs/dl_hw0/lib/python3.8/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=846335;file:///Users/Gautham/miniconda3/envs/dl_hw0/lib/python3.8/site-packages/soccerdata/_common.py#89\89]8;;\

INFO:root:Saving cached data to /Users/Gautham/soccerdata/data/MatchHistory
Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR',
       ...
       'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50', 'Unnamed: 51',
       'Unnamed: 52', 'Attendance', 'HHW', 'AHW', 'HO', 'AO'],
      dtype='object', length=161)
preprocess
create predictors
getting elo


[09/21/23 19:48:01] INFO     Saving cached data to /Users/Gautham/soccerdata/data/ClubElo             ]8;id=396922;file:///Users/Gautham/miniconda3/envs/dl_hw0/lib/python3.8/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=82627;file:///Users/Gautham/miniconda3/envs/dl_hw0/lib/python3.8/site-packages/soccerdata/_common.py#89\89]8;;\

INFO:root:Saving cached data to /Users/Gautham/soccerdata/data/ClubElo
hi Charlton
hi Man City
hi Chelsea
hi West Ham
hi Coventry
hi Middlesbrough
hi Southampton
hi Derby
hi Leeds
hi Everton
hi Leicester
hi Aston Villa
hi Bradford
hi Liverpool
hi Arsenal
hi Sunderland
hi Ipswich
hi Tottenham
hi Newcastle
hi Man United
hi Bolton
hi Blackburn
hi Fulham
hi West Brom
hi Birmingham
hi Portsmouth
hi Wolves
hi Norwich
hi Crystal Palace
hi Wigan
hi Watford
hi Reading
hi Sheffield United
hi Hull
hi Stoke
hi Burnley
hi Blackpool
hi QPR
hi Swansea
hi Cardiff
hi Bournemouth
hi Brighton
hi Huddersfield
hi Brentford
hi Forest
hi Luton
create rolling
predicting


In [925]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV
    

def random_forest(data, predictors, train, test, call_type):  
    model = ExtraTreesClassifier(bootstrap=True, criterion="entropy", max_features=0.8, min_samples_leaf=17, min_samples_split=5, n_estimators=100)

    #model = RandomForestClassifier(n_estimators=50, min_samples_split=7, random_state=1)  
    param_grid = {
	    "max_depth": [3],
	    "learning_rate": [0.01],
	    "gamma": [0, 0.25, 1],
	    "reg_lambda": [0, 1, 10],
	    "scale_pos_weight": [1, 3, 5],
	    "subsample": [0.8],
	    "colsample_bytree": [0.5],
	}
    params = {'colsample_bytree': 0.5, 'gamma': 1, 'learning_rate': 0.01, 'max_depth': 3, 'reg_lambda': 10, 'scale_pos_weight': 1, 'subsample': 0.8}
    
    model = xgb.XGBClassifier(**params, objective='binary:logistic')
    #model = xgb.XGBRegressor()
    #grid_cv = GridSearchCV(model, param_grid, n_jobs=-1, cv=3, scoring='roc_auc')
    #_ = grid_cv.fit(train[predictors], train['Target'])
    #print(grid_cv.best_score_, grid_cv.best_params_)

    #model.fit(train[predictors], train['Target'])
    model.fit(train[predictors], train['Target'])
    preds = model.predict(test[predictors])
    if call_type == 'predict':
        return preds
    #combined = pd.DataFrame(dict(actual=test['Target'], prediction=preds), index = test.index)
    precision = precision_score(test['Target'], preds)
    accuracy = accuracy_score(test['Target'], preds)
    combined = pd.DataFrame(dict(actual=test['Target'], prediction=preds), index = test.index)
    #precision = precision_score(test['TotalGoals'], preds)
    #accuracy = accuracy_score(test['TotalGoals'], preds)
    #precision, accuracy = 1,1
    #precision = cross_val_score(model, train[predictors], train['TotalGoals'], scoring='neg_mean_absolute_error')
    return combined, precision, accuracy

In [964]:
predictors = ['VenueCode', 'OpponentCode', 'ELODif']
#predictors = ['VenueCode', 'ELODif']
new_df = df_rolling.dropna(subset=['O2.5'])
new_df.shape
train = new_df[new_df['Date'] < '2016-08']
#test = new_df[new_df['Date'] >= '2022-08' and new_df['Date'] < '2022-08']
test = new_df.loc[(new_df['Date'] >= '2023-08')]# & (new_df['Date'] < '2022-05')]
combined, precision, accuracy = random_forest(df_rolling, predictors+new_cols, train, test, 'model')
precision, accuracy, combined
#(combined['actual'] - combined['prediction']).mean()

(0.7358490566037735,
 0.6395348837209303,
        actual  prediction
 16636       1           1
 16637       1           1
 16638       0           0
 16639       0           0
 16640       1           1
 ...       ...         ...
 16717       1           1
 16718       0           1
 16719       0           1
 16720       0           0
 16721       0           0
 
 [86 rows x 2 columns])

In [980]:
#train = new_df#[new_df['Date'] < '2023-08']
#test = new_df[new_df['Date'] >= '2022-08' and new_df['Date'] < '2022-08']
#test = 
#test = new_df.loc[(new_df['Date'] >= '2023-08')]# & (new_df['Date'] < '2022-05')]
#combined, precision, accuracy = random_forest(df_rolling, predictors+new_cols, train, test)
#schedule = get_schedule()
#schedule.loc[(schedule['Date'] == '23/09/23') | (schedule['Date'] == '24/09/23')]
combined
new_cols
df1
#df_rolling.loc[16636]

,GameID,Date,Team,Opponent,TotalGoals,TeamGoals,OpponentGoals,Result,Referee,TeamShots,OpponentShots,TeamShotsonTarget,OpponentShotsonTarget,TeamFouls,OpponentFouls,TeamCorners,OpponentCorners,TeamYellowCards,OpponentYellowCards,TeamRedCards,OpponentRedCards,O2.5,U2.5,TeamWinOdds,DrawOdds,OpponentWinOdds,Venue,VenueCode,OpponentCode,Target
0,0,2000-08-19,Charlton,Man City,4,4,0,W,Rob Harris,17,8,14,4,13,12,6,6,1,2,0,0,NaN,NaN,NaN,NaN,NaN,Home,1,27,1
1,0,2000-08-19,Man City,Charlton,4,0,4,L,Rob Harris,8,17,4,14,12,13,6,6,2,1,0,0,NaN,NaN,NaN,NaN,NaN,Away,0,12,1
2,1,2000-08-19,Chelsea,West Ham,6,4,2,W,Graham Barber,17,12,10,5,19,14,7,7,1,2,0,0,NaN,NaN,NaN,NaN,NaN,Home,1,43,1
3,1,2000-08-19,West Ham,Chelsea,6,2,4,L,Graham Barber,12,17,5,10,14,19,7,7,2,1,0,0,NaN,NaN,NaN,NaN,NaN,Away,0,13,1
4,2,2000-08-19,Coventry,Middlesbrough,4,1,3,L,Barry Knight,6,16,3,9,15,21,8,4,5,3,1,0,NaN,NaN,NaN,NaN,NaN,Home,1,29,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16893,8446,2023-09-17,Arsenal,Everton,1,1,0,W,S Hooper,13,8,4,1,10,12,11,1,1,1,0,0,1.67,2.23,1.50,4.50,6.00,Away,0,17,0
16894,8447,2023-09-17,Bournemouth,Chelsea,0,0,0,D,D Coote,13,14,4,6,15,20,1,7,1,5,0,0,1.67,2.19,4.33,3.75,1.80,Home,1,13,0
16895,8447,2023-09-17,Chelsea,Bournemouth,0,0,0,D,D Coote,14,13,6,4,20,15,7,1,5,1,0,0,1.67,2.19,1.80,3.75,4.33,Away,0,6,0
16896,8448,2023-09-18,Forest,Burnley,2,1,1,D,R Jones,14,10,4,3,16,10,5,4,2,3,0,1,1.92,1.87,2.20,3.50,3.25,Home,1,10,0


In [958]:
c1 = combined.merge(df_rolling[['Date', 'Team', 'Opponent', 'TotalGoals', 'GameID']], left_index=True, right_index=True)
c1
#list(c1.groupby(['Team', 'Opponent']))

,actual,prediction,Date,Team,Opponent,TotalGoals,GameID
16636,1,1,2023-08-11,Burnley,Man City,3,8400
16637,1,1,2023-08-11,Man City,Burnley,3,8400
16638,0,0,2023-08-12,West Ham,Bournemouth,2,8402
16639,0,0,2023-08-12,Bournemouth,West Ham,2,8402
16640,1,1,2023-08-12,Brighton,Luton,5,8403
...,...,...,...,...,...,...,...
16717,1,1,2023-09-16,Liverpool,Wolves,4,8445
16718,0,1,2023-09-17,Everton,Arsenal,1,8446
16719,0,1,2023-09-17,Arsenal,Everton,1,8446
16720,0,0,2023-09-17,Bournemouth,Chelsea,0,8447


In [965]:
# predictions where both home and away team prediction match
predictions = c1.groupby('GameID').filter(lambda x: x['prediction'].sum()!=1)
#predictions = c1.groupby('GameID')[['Team', 'Opponent','actual', 'prediction']].sum()
#predictions[['actual','prediction']] = predictions[['actual','prediction']] / 2 .filter(lambda x: x['prediction'].sum()!=1)#[['GameID','Date', 'prediction','actual']]
#predictions = predictions[predictions['Date'] > '2023-08']

pred_with_odds = predictions.merge(df1[['GameID','O2.5', 'U2.5']], how='inner',on='GameID').drop_duplicates()

pred_with_odds = pred_with_odds[['GameID', 'O2.5', 'U2.5', 'prediction', 'actual']].drop_duplicates()
#pred_with_odds[pred_with_odds['actual'] == pred_with_odds['prediction']].shape[0] / pred_with_odds.shape[0] 

pred_with_odds

#predictions
#[pred_with_odds['prediction'] != pred_with_odds['actual']]

,GameID,O2.5,U2.5,prediction,actual
0,8400,1.67,2.28,1,1
4,8402,1.90,2.12,0,0
8,8404,2.01,1.71,0,0
12,8406,2.20,1.53,0,0
16,8407,1.67,2.52,1,0
...,...,...,...,...,...
104,8443,1.50,2.97,1,1
108,8444,1.62,2.08,1,1
112,8445,1.44,2.56,1,1
116,8446,1.67,2.23,1,0


In [966]:
def calculate_profit(df, odds_over=[], odds_under=[], bets=[], actual=[]):
    odds_over = list(df['O2.5'])
    odds_under = list(df['U2.5'])
    bets = list(df['prediction'])
    actual = list(df['actual'])
    games = df['GameID']
    american_odds = []
    profits = []
    profit = 0
    bets1 = []
    print(odds_over, odds_under)
    for i,bet in enumerate(bets):
        if bet == 0:
            value = odds_under[i]
        else:
            value = odds_over[i]
        if value > 2.00:
            american = (value - 1) * 100
            if bet == actual[i]:
                profit = value * 100
                bet = 100
            else:
                profit = -100
                bet = 100
        else:
            american = -100 / (value - 1)
            if bet == actual[i]:
                bet = value * 100
                bet = 100
                profit = 100 * value - bet
                #profit = 100
                
            else:
                profit = -value * 100
                profit = -100
                #bet = value * 100
                bet = 100
                
            #profit = value * 100 * actual[i] - value*100
        bets1.append(bet)
        profits.append(profit)
        american_odds.append(american)       

    return pd.DataFrame(zip(*[games, profits, bets1])), american_odds

In [967]:
df, america = calculate_profit(pred_with_odds)
df.sum()

[1.67, 1.9, 2.01, 2.2, 1.67, 1.73, 1.96, 1.33, 1.73, 1.57, 1.92, 1.94, 2.1, 1.44, 1.86, 1.57, 1.87, 1.73, 1.5, 1.4, 2.2, 1.4, 1.57, 2.2, 1.89, 1.44, 1.5, 1.62, 1.44, 1.67, 1.67] [2.28, 2.12, 1.71, 1.53, 2.52, 2.15, 1.73, 3.77, 2.04, 2.54, 2.09, 1.8, 1.88, 2.58, 2.06, 2.36, 2.16, 2.06, 2.7, 2.89, 1.82, 2.97, 2.71, 1.78, 1.77, 2.88, 2.97, 2.08, 2.56, 2.23, 2.19]


0    261186.0
1       688.0
2      3100.0
dtype: float64

In [ ]:
'''
from tpot import TPOTClassifier
#from sklearn.datasets import load_digits
#from sklearn.model_selection import train_test_split

#digits = load_digits()

#target = total_df['TotalGoals']
#X_train, X_test, y_train, y_test = train_test_split(data, target, train_size=0.75, test_size=0.25)

X_train = train.loc[:, predictors+new_cols]
y_train = train.loc[:, 'Target']

X_test = test.loc[:, predictors+new_cols]
y_test = test.loc[:, 'Target']

pipeline_optimizer = TPOTClassifier(generations=1, population_size=100, cv=5,
                                    random_state=42, verbosity=2)
pipeline_optimizer.fit(X_train, y_train)
print(pipeline_optimizer.score(X_test, y_test))
pipeline_optimizer.export('tpot_exported_pipeline.py')
'''



In [729]:
def predict(train, test, dates):
    rolling = pd.read_csv('cache/rolling.csv')
    elo = pd.read_csv('cache/elo.csv')
    schedule = pd.read_csv('cache/schedule.csv')
    #print(rolling['Date'])
    recent_rolling = rolling[rolling['Date'] >= '2023-09-16']
    #schedule.loc[(schedule['Date'] == '23/09/23') | (schedule['Date'] == '24/09/23')]
    #print(recent_rolling)
    sched = pd.DataFrame()
    for date in dates:
        sched = pd.concat([sched, schedule.loc[(schedule['Date'] == date)]])
    #print(sched)
    sched = home_away(sched)

    #print(sched)
    sched = sched.merge(recent_rolling[['Team', 'TeamELO']], on='Team', how='inner')
    sched = sched.merge(recent_rolling[['Opponent', 'OpponentELO']], on='Opponent', how='inner')
    sched['ELODif'] = sched['TeamELO'] - sched['OpponentELO']
    
    new = pd.concat([elo, sched[['Date', 'Team', 'Opponent', 'TeamELO', 'OpponentELO', 'ELODif', 'Venue', "GameID"]]])

    #return new
    new = create_predictors(new)
    new = create_rolling(new)
    
    predictors = ['VenueCode', 'OpponentCode', 'ELODif']
    #predictors = ['VenueCode', 'ELODif']
    #print(new)
    new_df = new.dropna(subset=['O2.5'])
    train = new_df[new_df['Date'] < '2023-09-18']
    #return train
    #test = new_df[new_df['Date'] >= '2022-08' and new_df['Date'] < '2022-08']
    test = new.loc[(new['Date'] == '23/09/23') | (new['Date'] == '24/09/23')]
    preds = random_forest(df_rolling, predictors+new_cols, train, test, 'predict')
    #c1 = preds.merge(test[['Date', 'Team', 'Opponent', 'TotalGoals', 'GameID']], left_index=True, right_index=True)

    return preds, test

preds, c1 = predict([],[],['23/09/23', '24/09/23'])
c1['preds'] = preds
c1[['Team', 'Opponent', 'preds']]
c1.sort_values(['GameID'])[['Date','Team', 'Opponent', 'preds', 'GameID']]
#c1.sort_index()[['Date','Team', 'Opponent', 'preds']]


,Date,Team,Opponent,preds,GameID
0,23/09/23,Brentford,Everton,0,0
1,23/09/23,Everton,Brentford,0,0
2,23/09/23,Fulham,Crystal Palace,0,2
3,23/09/23,Crystal Palace,Fulham,0,2
4,23/09/23,Luton,Wolves,0,3
5,23/09/23,Wolves,Luton,1,3
6,24/09/23,Tottenham,Arsenal,1,5
7,24/09/23,Arsenal,Tottenham,1,5
8,24/09/23,Brighton,Bournemouth,1,6
9,24/09/23,Bournemouth,Brighton,1,6


In [1071]:
df_rolling = pd.read_csv('/Users/Gautham/Projects/Betting/cache/rolling_dif.csv')
new_df = df_rolling.dropna(subset=['O2.5'])
predictors = ['VenueCode', 'OpponentCode', 'ELODif'] 
new_cols = ['GoalsRolling_Dif',
 'ShotsRolling_Dif',
 'ShotsonTargetRolling_Dif',
 'CornersRolling_Dif']
def split_data(new_df, drop_cols, pred_cols):

    new_df = new_df.drop(columns=drop_cols, axis=1)
    train = new_df[new_df['Date'] < '2018-08']
    #test = new_df[new_df['Date'] >= '2022-08' and new_df['Date'] < '2022-08']
    test = new_df.loc[(new_df['Date'] >= '2018-08')]
    train = train.select_dtypes(include=np.number)
    test = test.select_dtypes(include=np.number)
    X_train = train.loc[:, pred_cols]#predictors+new_cols]
    y_train = train.loc[:, 'Target']

    X_test = test.loc[:, pred_cols]
    y_test = test.loc[:, 'Target']
    return X_train, y_train, X_test, y_test


In [1052]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

df_rolling.select_dtypes(include=np.number)

,Unnamed: 0.1,Unnamed: 0,GameID,TotalGoals,TeamGoals,OpponentGoals,TeamShots,OpponentShots,TeamShotsonTarget,OpponentShotsonTarget,TeamFouls,OpponentFouls,TeamCorners,OpponentCorners,TeamYellowCards,OpponentYellowCards,TeamRedCards,OpponentRedCards,O2.5,U2.5,TeamWinOdds,DrawOdds,OpponentWinOdds,VenueCode,OpponentCode,Target,TeamELO,OpponentELO,ELODif,TotalGoalsRolling,TeamGoalsRolling,OpponentGoalsRolling,TeamShotsRolling,OpponentShotsRolling,TeamShotsonTargetRolling,OpponentShotsonTargetRolling,TeamCornersRolling,OpponentCornersRolling,WinTarget,GoalsRolling_Dif,ShotsRolling_Dif,ShotsonTargetRolling_Dif,CornersRolling_Dif
0,0,0,19,6,3,3,14,9,7,4,7,6,7,1,0,0,0,0,NaN,NaN,NaN,NaN,NaN,1,25,1,1625.633423,1770.309570,-144.676147,3.500000,1.500000,2.000000,12.500000,6.500000,5.000000,4.500000,7.000000,5.000000,0,-0.500000,6.000000,0.500000,2.000000
1,1,1,19,6,3,3,9,14,4,7,6,7,1,7,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,36,1,1770.309570,1625.633423,144.676147,1.500000,0.500000,1.000000,11.500000,10.000000,7.000000,7.000000,8.500000,5.500000,0,-0.500000,1.500000,0.000000,3.000000
2,2,2,20,8,5,3,18,7,9,4,12,15,8,3,0,1,0,0,NaN,NaN,NaN,NaN,NaN,1,12,1,1870.864136,1615.091187,255.772949,1.500000,1.000000,0.500000,15.500000,7.500000,9.500000,3.000000,9.500000,6.500000,2,0.500000,8.000000,6.500000,3.000000
3,3,3,20,8,3,5,7,18,4,9,15,12,3,8,1,0,0,0,NaN,NaN,NaN,NaN,NaN,0,0,1,1615.091187,1870.864136,-255.772949,3.500000,2.000000,1.500000,12.500000,10.500000,9.000000,6.000000,5.500000,4.500000,1,0.500000,2.000000,3.000000,1.000000
4,4,4,21,0,0,0,8,13,4,8,11,12,6,8,1,2,0,0,NaN,NaN,NaN,NaN,NaN,1,24,0,1596.314331,1699.908447,-103.594116,1.500000,1.000000,0.500000,7.500000,15.000000,2.500000,8.000000,3.500000,5.000000,0,0.500000,-7.500000,-5.500000,-1.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16717,16717,16717,8445,4,3,1,16,11,6,2,4,10,4,4,3,1,0,0,1.44,2.56,1.45,5.00,6.00,0,45,1,1961.335938,1708.177490,253.158447,3.333333,2.666667,0.666667,17.333333,15.000000,6.000000,5.333333,8.666667,3.333333,2,2.000000,2.333333,0.666667,5.333333
16718,16718,16718,8446,1,0,1,8,13,1,4,12,10,1,11,1,1,0,0,1.67,2.23,6.00,4.50,1.50,1,0,0,1682.139038,1928.753418,-246.614380,3.000000,0.666667,2.333333,13.333333,12.333333,5.000000,5.666667,6.333333,3.666667,1,-1.666667,1.000000,-0.666667,2.666667
16719,16719,16719,8446,1,1,0,13,8,4,1,10,12,11,1,1,1,0,0,1.67,2.23,1.50,4.50,6.00,0,17,0,1928.753418,1682.139038,246.614380,3.000000,2.000000,1.000000,16.666667,10.666667,6.333333,2.333333,9.333333,2.333333,2,1.000000,6.000000,4.000000,7.000000
16720,16720,16720,8447,0,0,0,13,14,4,6,15,20,1,7,1,5,0,0,1.67,2.19,4.33,3.75,1.80,1,13,0,1659.008789,1774.225342,-115.216553,3.333333,1.000000,2.333333,12.000000,21.333333,4.333333,7.666667,3.333333,5.666667,0,-1.333333,-9.333333,-3.333333,-2.333333


In [1128]:
def get_redundant_pairs(df):
   '''Get diagonal and lower triangular pairs of correlation matrix'''
   pairs_to_drop = set()
   cols = df.columns
   for i in range(0, df.shape[1]):
       for j in range(0, i + 1):
           pairs_to_drop.add((cols[i], cols[j]))
   return pairs_to_drop

def get_top_abs_correlations(df, n=2000):
   au_corr = df.corr().abs().unstack()
   labels_to_drop = get_redundant_pairs(df)
   au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
   return au_corr

def get_corr_features(df, corr_threshold=.95):
   fsdf = df.select_dtypes(exclude=['object'])
   #fsdf = fsdf.filter(like='precomp')
   print("Top Absolute Correlations")
   x = get_top_abs_correlations(fsdf, n=2000)
   #print(dict(x))
   drop_features = []
   for idx, val in x.items():
       if val > corr_threshold:
           drop_features.append(idx[1])
   drop_features = list(set(drop_features)) # remove dupes
   return drop_features
df_rolling = pd.read_csv('/Users/Gautham/Projects/Betting/cache/rolling_dif.csv')
new_df = df_rolling.dropna(subset=['O2.5'])
new_df = new_df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0', 'O2.5', 'U2.5'])
drop_features = get_corr_features(new_df, corr_threshold=.95)
new_df = new_df.drop(drop_features, axis=1)
new_df

Top Absolute Correlations


,GameID,Date,Team,Opponent,TotalGoals,TeamGoals,OpponentGoals,Result,Referee,TeamShots,OpponentShots,TeamShotsonTarget,OpponentShotsonTarget,TeamFouls,OpponentFouls,TeamCorners,OpponentCorners,TeamYellowCards,OpponentYellowCards,TeamRedCards,OpponentRedCards,TeamWinOdds,DrawOdds,OpponentWinOdds,Venue,VenueCode,OpponentCode,Target,TeamELO,OpponentELO,ELODif,TotalGoalsRolling,TeamGoalsRolling,OpponentGoalsRolling,TeamShotsRolling,OpponentShotsRolling,TeamShotsonTargetRolling,OpponentShotsonTargetRolling,TeamCornersRolling,OpponentCornersRolling,WinTarget,GoalsRolling_Dif,ShotsRolling_Dif,ShotsonTargetRolling_Dif,CornersRolling_Dif
1728,889,2002-11-16,Arsenal,Tottenham,3,3,0,W,M Riley,17,6,12,5,15,10,3,1,0,2,0,1,1.533,3.50,5.500,Home,1,40,1,1886.304565,1673.461426,212.843140,1.666667,1.000000,0.666667,13.666667,3.666667,7.333333,2.666667,10.333333,4.333333,2,0.333333,10.000000,4.666667,6.000000
1729,889,2002-11-16,Tottenham,Arsenal,3,0,3,L,M Riley,6,17,5,12,10,15,1,3,2,0,1,0,5.500,3.50,1.533,Away,0,0,1,1673.461426,1886.304565,-212.843140,1.666667,0.333333,1.333333,11.333333,12.000000,6.000000,4.666667,6.666667,3.666667,1,-1.000000,-0.666667,1.333333,3.000000
1730,890,2002-11-16,Middlesbrough,Chelsea,1,0,1,L,P Dowd,7,13,4,5,11,9,7,5,1,2,0,0,4.500,3.20,1.727,Away,0,13,0,1687.139404,1775.028809,-87.889404,2.333333,1.000000,1.333333,13.000000,12.666667,6.333333,9.000000,7.000000,3.666667,1,-0.333333,0.333333,-2.666667,3.333333
1731,890,2002-11-16,Chelsea,Middlesbrough,1,1,0,W,P Dowd,13,7,5,4,9,11,5,7,2,1,0,0,1.727,3.20,4.500,Home,1,29,0,1775.028809,1687.139404,87.889404,1.666667,1.666667,0.000000,13.333333,9.666667,6.333333,4.666667,4.666667,5.333333,2,1.666667,3.666667,1.666667,-0.666667
1732,891,2002-11-16,Charlton,Man City,1,1,0,W,G Barber,13,15,4,4,16,10,5,7,4,0,0,0,4.500,3.40,1.667,Away,0,27,0,1626.263794,1664.720215,-38.456421,2.000000,0.666667,1.333333,10.000000,12.000000,6.000000,7.000000,4.666667,9.666667,2,-0.666667,-2.000000,-1.000000,-5.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16717,8445,2023-09-16,Liverpool,Wolves,4,3,1,W,M Oliver,16,11,6,2,4,10,4,4,3,1,0,0,1.450,5.00,6.000,Away,0,45,1,1961.335938,1708.177490,253.158447,3.333333,2.666667,0.666667,17.333333,15.000000,6.000000,5.333333,8.666667,3.333333,2,2.000000,2.333333,0.666667,5.333333
16718,8446,2023-09-17,Everton,Arsenal,1,0,1,L,S Hooper,8,13,1,4,12,10,1,11,1,1,0,0,6.000,4.50,1.500,Home,1,0,0,1682.139038,1928.753418,-246.614380,3.000000,0.666667,2.333333,13.333333,12.333333,5.000000,5.666667,6.333333,3.666667,1,-1.666667,1.000000,-0.666667,2.666667
16719,8446,2023-09-17,Arsenal,Everton,1,1,0,W,S Hooper,13,8,4,1,10,12,11,1,1,1,0,0,1.500,4.50,6.000,Away,0,17,0,1928.753418,1682.139038,246.614380,3.000000,2.000000,1.000000,16.666667,10.666667,6.333333,2.333333,9.333333,2.333333,2,1.000000,6.000000,4.000000,7.000000
16720,8447,2023-09-17,Bournemouth,Chelsea,0,0,0,D,D Coote,13,14,4,6,15,20,1,7,1,5,0,0,4.330,3.75,1.800,Home,1,13,0,1659.008789,1774.225342,-115.216553,3.333333,1.000000,2.333333,12.000000,21.333333,4.333333,7.666667,3.333333,5.666667,0,-1.333333,-9.333333,-3.333333,-2.333333


In [1130]:
def xgb_model(X_train, y_train, X_test, y_test):
    params = {'colsample_bytree': 0.5, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 5, 'reg_lambda': 0, 'subsample': 0.8}
    clf = xgb.XGBClassifier(**params)
    '''verbosity=0,
                           learning_rate=0.01,
                           colsample_bytree=0.7,
                           subsample=0.7,
                           objective='binary:logistic',
                           n_estimators=200,
                           max_depth=3,
                           gamma=0.1,
                           seed=1,
                           eta=0.2,
                           min_child_weight=2)
    '''
    clf.fit(X_train, y_train)
    # Accuracy
    ypred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, ypred)
    return clf, accuracy

def print_feature_importances(clf, cols, print_num):
   feats = {}  # a dict to hold feature_name: feature_importance
   for feature, importance in zip(cols, clf.feature_importances_):
       feats[feature] = importance
   q = [[k, v] for k, v in sorted(feats.items(), key=lambda item: item[1])]
   for x in q[-print_num:]:
       print(x[0], x[1])

print_num = 30
cols = X_train.columns
clf, accuracy = xgb_model(X_train, y_train, X_test, y_test)
print_feature_importances(clf, cols, print_num)
print(accuracy)

CornersRolling_Dif 0.044304173
TotalGoalsRolling 0.04782097
TeamGoalsRolling 0.04833898
GoalsRolling_Dif 0.04950154
OpponentGoalsRolling 0.050187998
OpponentCornersRolling 0.051884867
ShotsonTargetRolling_Dif 0.051916838
OpponentCode 0.052513953
OpponentShotsonTargetRolling 0.052755304
TeamCornersRolling 0.053894848
OpponentShotsRolling 0.05476493
ShotsRolling_Dif 0.055347633
TeamShotsonTargetRolling 0.05962794
OpponentELO 0.06007228
TeamELO 0.060864836
VenueCode 0.064408995
ELODif 0.06911364
TeamShotsRolling 0.072680265
0.555662188099808


In [1093]:
get_corr_features(X_train, corr_threshold=.80)

Top Absolute Correlations
{('ShotsRolling_Dif', 'ShotsonTargetRolling_Dif'): 0.8618784084372058, ('TeamShotsRolling', 'ShotsRolling_Dif'): 0.8406512917055434, ('OpponentShotsRolling', 'ShotsRolling_Dif'): 0.8355428184415378, ('OpponentCornersRolling', 'CornersRolling_Dif'): 0.8102743909409199, ('TeamCornersRolling', 'CornersRolling_Dif'): 0.8040778706622487, ('TeamGoalsRolling', 'GoalsRolling_Dif'): 0.770203739516636, ('OpponentGoalsRolling', 'GoalsRolling_Dif'): 0.7522841225979943, ('TeamShotsRolling', 'ShotsonTargetRolling_Dif'): 0.7280329215005912, ('TeamShotsonTargetRolling', 'ShotsonTargetRolling_Dif'): 0.7231982441240634, ('OpponentShotsRolling', 'ShotsonTargetRolling_Dif'): 0.7165919591612633, ('OpponentELO', 'ELODif'): 0.7126129084695421, ('TeamELO', 'ELODif'): 0.7119521397470606, ('OpponentShotsonTargetRolling', 'ShotsonTargetRolling_Dif'): 0.7019426179822543, ('TeamShotsRolling', 'TeamShotsonTargetRolling'): 0.683632416534644, ('ShotsRolling_Dif', 'CornersRolling_Dif'): 0.663

['ShotsonTargetRolling_Dif', 'ShotsRolling_Dif', 'CornersRolling_Dif']

In [1094]:
new_df.select_dtypes(include=np.number).columns

Index(['GameID', 'TotalGoals', 'TeamGoals', 'OpponentGoals', 'TeamShots',
       'OpponentShots', 'TeamShotsonTarget', 'OpponentShotsonTarget',
       'TeamFouls', 'OpponentFouls', 'TeamCorners', 'OpponentCorners',
       'TeamYellowCards', 'OpponentYellowCards', 'TeamRedCards',
       'OpponentRedCards', 'TeamWinOdds', 'DrawOdds', 'OpponentWinOdds',
       'VenueCode', 'OpponentCode', 'Target', 'TeamELO', 'OpponentELO',
       'ELODif', 'TotalGoalsRolling', 'TeamGoalsRolling',
       'OpponentGoalsRolling', 'TeamShotsRolling', 'OpponentShotsRolling',
       'TeamShotsonTargetRolling', 'OpponentShotsonTargetRolling',
       'TeamCornersRolling', 'OpponentCornersRolling', 'WinTarget',
       'GoalsRolling_Dif', 'ShotsRolling_Dif', 'ShotsonTargetRolling_Dif',
       'CornersRolling_Dif'],
      dtype='object')

In [1129]:
drop = []#['Unnamed: 0.1', 'Unnamed: 0', 'GameID']
pred = ['VenueCode', 'OpponentCode', 'TeamELO', 'OpponentELO', 'ELODif', 'TotalGoalsRolling',
       'TeamGoalsRolling', 'OpponentGoalsRolling', 'TeamShotsRolling',
       'OpponentShotsRolling', 'TeamShotsonTargetRolling',
       'OpponentShotsonTargetRolling', 'TeamCornersRolling',
       'OpponentCornersRolling', 'GoalsRolling_Dif',
       'ShotsRolling_Dif', 'ShotsonTargetRolling_Dif', 'CornersRolling_Dif'] 
'''
#pred = ['VenueCode', 'OpponentCode', 'TeamELO', 'OpponentELO', 'ELODif', 'TotalGoalsRolling',
       'TeamGoalsRolling', 'OpponentGoalsRolling', 'TeamShotsRolling',
       'OpponentShotsRolling', 'TeamShotsonTargetRolling',
       'OpponentShotsonTargetRolling', 'TeamCornersRolling',
       'OpponentCornersRolling', 'GoalsRolling_Dif',
        ] 
'''
       
  #     ['ShotsonTargetRolling_Dif', 'ShotsRolling_Dif', 'CornersRolling_Dif']
X_train, y_train, X_test, y_test = split_data(new_df, drop, pred)
X_train

,VenueCode,OpponentCode,TeamELO,OpponentELO,ELODif,TotalGoalsRolling,TeamGoalsRolling,OpponentGoalsRolling,TeamShotsRolling,OpponentShotsRolling,TeamShotsonTargetRolling,OpponentShotsonTargetRolling,TeamCornersRolling,OpponentCornersRolling,GoalsRolling_Dif,ShotsRolling_Dif,ShotsonTargetRolling_Dif,CornersRolling_Dif
1728,1,40,1886.304565,1673.461426,212.843140,1.666667,1.000000,0.666667,13.666667,3.666667,7.333333,2.666667,10.333333,4.333333,0.333333,10.000000,4.666667,6.000000
1729,0,0,1673.461426,1886.304565,-212.843140,1.666667,0.333333,1.333333,11.333333,12.000000,6.000000,4.666667,6.666667,3.666667,-1.000000,-0.666667,1.333333,3.000000
1730,0,13,1687.139404,1775.028809,-87.889404,2.333333,1.000000,1.333333,13.000000,12.666667,6.333333,9.000000,7.000000,3.666667,-0.333333,0.333333,-2.666667,3.333333
1731,1,29,1775.028809,1687.139404,87.889404,1.666667,1.666667,0.000000,13.333333,9.666667,6.333333,4.666667,4.666667,5.333333,1.666667,3.666667,1.666667,-0.666667
1732,0,27,1626.263794,1664.720215,-38.456421,2.000000,0.666667,1.333333,10.000000,12.000000,6.000000,7.000000,4.666667,9.666667,-0.666667,-2.000000,-1.000000,-5.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13589,0,39,1620.356567,1630.464355,-10.107788,1.666667,0.666667,1.000000,6.666667,16.333333,2.000000,3.666667,2.000000,8.000000,-0.333333,-9.666667,-1.666667,-6.000000
13590,1,24,1913.709106,1700.989502,212.719605,1.333333,1.000000,0.333333,14.000000,12.000000,5.000000,3.000000,6.333333,3.000000,0.666667,2.000000,2.000000,3.333333
13591,0,40,1700.989502,1913.709106,-212.719605,3.666667,1.000000,2.666667,12.666667,14.000000,4.333333,6.333333,5.333333,5.666667,-1.666667,-1.333333,-2.000000,-0.333333
13592,1,17,1664.282593,1711.364380,-47.081787,2.333333,1.000000,1.333333,8.666667,16.000000,2.000000,5.000000,1.333333,7.000000,-0.333333,-7.333333,-3.000000,-5.666667


In [1135]:
import xgboost as xgb
from sklearn.feature_selection import SelectFromModel


def threshold_selector(clf, X_train, y_train, X_test, y_test):
   thresholds = np.arange(.1, 5, .05)
   avgs = []
   loopavg = []
   count = 1

   for thresh in thresholds:
       # Run each threshold 10x for the avg
       for x in range(0, 10):
           selection = SelectFromModel(clf, threshold=f'{thresh}*median', prefit=True)
           select_X_train = selection.transform(X_train)
           model = xgb.XGBClassifier(verbosity=0,
                                reg_lambda=0.023385762997113632,
                                reg_alpha=0.003694895205081855,
                                #tree_method="gpu_hist",
                                objective="binary:logistic",
                                n_jobs=-1,
                                learning_rate=0.0059107879099318415,
                                min_child_weight=15,
                                max_depth=1,
                                max_delta_step=10,
                                subsample=0.5370056644955932,
                                colsample_bytree=0.5742787613391558,
                                gamma=0.09815563994539223,
                                n_estimators=143,
                                eta=0.1134711359195081,
                                seed=1)
           count +=1
           model.fit(select_X_train, y_train.values.ravel())
           select_X_test = selection.transform(X_test)
           ypred = model.predict(select_X_test)
           accuracy = accuracy_score(y_test, ypred)
           msg = f"Thresh={thresh}, n={select_X_train.shape[1]}, Accuracy: {accuracy * 100.0}"
           print(msg)
           loopavg.append(accuracy)

       avgs.append((thresh, np.mean(loopavg)))
       loopavg = []

   for x in avgs:
       print(x)

threshold_selector(clf, X_train, y_train, X_test, y_test)

Thresh=0.1, n=18, Accuracy: 55.91810620601407
Thresh=0.1, n=18, Accuracy: 55.91810620601407
Thresh=0.1, n=18, Accuracy: 55.91810620601407
Thresh=0.1, n=18, Accuracy: 55.91810620601407
Thresh=0.1, n=18, Accuracy: 55.91810620601407
Thresh=0.1, n=18, Accuracy: 55.91810620601407
Thresh=0.1, n=18, Accuracy: 55.91810620601407
Thresh=0.1, n=18, Accuracy: 55.91810620601407
Thresh=0.1, n=18, Accuracy: 55.91810620601407
Thresh=0.1, n=18, Accuracy: 55.91810620601407
Thresh=0.15000000000000002, n=18, Accuracy: 55.91810620601407
Thresh=0.15000000000000002, n=18, Accuracy: 55.91810620601407
Thresh=0.15000000000000002, n=18, Accuracy: 55.91810620601407
Thresh=0.15000000000000002, n=18, Accuracy: 55.91810620601407
Thresh=0.15000000000000002, n=18, Accuracy: 55.91810620601407
Thresh=0.15000000000000002, n=18, Accuracy: 55.91810620601407
Thresh=0.15000000000000002, n=18, Accuracy: 55.91810620601407
Thresh=0.15000000000000002, n=18, Accuracy: 55.91810620601407
Thresh=0.15000000000000002, n=18, Accuracy: 

XGBoostError: [17:36:35] /Users/runner/work/xgboost/xgboost/src/data/iterative_dmatrix.cc:202: Check failed: n_features >= 1 (0 vs. 1) : Data must has at least 1 column.
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x00000001b1ef4ab5 dmlc::LogMessageFatal::~LogMessageFatal() + 117
  [bt] (1) 2   libxgboost.dylib                    0x00000001b20074ff xgboost::data::IterativeDMatrix::InitFromCPU(xgboost::Context const*, xgboost::BatchParam const&, void*, float, std::__1::shared_ptr<xgboost::DMatrix>) + 8431
  [bt] (2) 3   libxgboost.dylib                    0x00000001b2004d0c xgboost::data::IterativeDMatrix::IterativeDMatrix(void*, void*, std::__1::shared_ptr<xgboost::DMatrix>, void (*)(void*), int (*)(void*), float, int, int) + 1356
  [bt] (3) 4   libxgboost.dylib                    0x00000001b1fb9579 xgboost::DMatrix* xgboost::DMatrix::Create<void*, void*, void (void*), int (void*)>(void*, void*, std::__1::shared_ptr<xgboost::DMatrix>, void (*)(void*), int (*)(void*)

In [24]:
clubelo = sd.ClubElo('GER-Bundesliga', '2223')
#clubelo = clubelo.read_team_history('Düsseldorf')
#clubelo
v = clubelo.read_by_date('2023-11-08')
v[v['country'] == 'GER']#.team

[10/08/23 15:28:05] INFO     Saving cached data to /Users/Gautham/soccerdata/data/ClubElo             ]8;id=293105;file:///Users/Gautham/miniconda3/envs/dl_hw0/lib/python3.8/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=403205;file:///Users/Gautham/miniconda3/envs/dl_hw0/lib/python3.8/site-packages/soccerdata/_common.py#89\89]8;;\

,rank,country,level,elo,from,to,league
team,,,,,,,
Bayern,4.0,GER,1,1940.729004,2023-10-27,2023-12-31,GER-Bundesliga
RB Leipzig,14.0,GER,1,1838.462402,2023-10-27,2023-12-31,GER-Bundesliga
Dortmund,15.0,GER,1,1833.151855,2023-10-27,2023-12-31,GER-Bundesliga
Leverkusen,23.0,GER,1,1793.137573,2023-10-27,2023-12-31,GER-Bundesliga
Wolfsburg,46.0,GER,1,1716.327759,2023-10-27,2023-12-31,GER-Bundesliga
...,...,...,...,...,...,...,...
Rostock,NaN,GER,2,1403.596436,2023-10-27,2023-12-31,NaN
Elversberg,NaN,GER,2,1369.374268,2023-10-27,2023-12-31,NaN
Braunschweig,NaN,GER,2,1359.264282,2023-10-27,2023-12-31,NaN


In [25]:
list(v[v['country'] == 'GER'].reset_index().team)

['Bayern',
 'RB Leipzig',
 'Dortmund',
 'Leverkusen',
 'Wolfsburg',
 'Freiburg',
 'Frankfurt',
 'Union Berlin',
 'Stuttgart',
 'Hoffenheim',
 'Gladbach',
 'Mainz',
 'Koeln',
 'Werder',
 'Bochum',
 'Hamburg',
 'Augsburg',
 'Heidenheim',
 'St Pauli',
 'Duesseldorf',
 'Darmstadt',
 'Hertha',
 'Schalke',
 'Paderborn',
 'Holstein',
 'Hannover',
 'Fuerth',
 'Karlsruhe',
 'Magdeburg',
 'Lautern',
 'Nuernberg',
 'Rostock',
 'Elversberg',
 'Braunschweig',
 'Wehen',
 'Osnabrueck']

In [37]:
lo#['home_team'].unique()

,rank,team,country,level,elo,to
from,,,,,,
1948-07-04,NaN,Verona,ITA,2,1471.397217,1948-09-19
1948-09-20,NaN,Verona,ITA,2,1467.319214,1948-09-26
1948-09-27,NaN,Verona,ITA,2,1471.588501,1948-10-03
1948-10-04,NaN,Verona,ITA,2,1466.306152,1948-10-10
1948-10-11,NaN,Verona,ITA,2,1471.820801,1948-10-17
...,...,...,...,...,...,...
2023-09-22,94.0,Verona,ITA,1,1623.596191,2023-09-23
2023-09-24,95.0,Verona,ITA,1,1621.083252,2023-09-27
2023-09-28,95.0,Verona,ITA,1,1621.083252,2023-10-02


In [47]:
! pip install understatapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.2/397.2 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached chardet-4.0.0-py2.py3-none-any.whl (178 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.1/138.1 kB 5.1 MB/s eta 0:00:00
  Using cached requests-2.25.1-py2.py3-none-any.whl (61 kB)
  Using cached idna-2.10-py2.py3-none-any.whl (58 kB)
  Using cached certifi-2020.12.5-py2.py3-none-any.whl (147 kB)
  Created wheel for understatapi: filename=understatapi-0.6.1-py3-none-any.whl size=15008 sha256=31ef9491e17f34f641ab77474bfef7fad7ee92662f2f161108ef2669937397e5
  Stored in directory: /Users/Gautham/Library/Caches/pip/wheels/d1/6c/3a/269f970838b6faad43755592efb08e3d545902d0cd509b7c22
Successfully built understatapi
  Attempting uninstall: certifi
    Found existing installation: certifi 2023.7.22
    Uninstalling certif

In [74]:
processed = pd.read_csv('cache/PremierLeague/processed.csv')
processed[processed['Date'].str.contains('2022')]['Team'].unique()

array(['Crystal Palace', 'West Ham', 'Arsenal', 'Man City', 'Tottenham',
       'Watford', 'Aston Villa', 'Brentford', 'Brighton', 'Everton',
       'Burnley', 'Leeds', 'Chelsea', 'Liverpool', 'Man United', 'Wolves',
       'Southampton', 'Norwich', 'Newcastle', 'Leicester', 'Forest',
       'Bournemouth', 'Fulham'], dtype=object)

In [126]:
t = understat.league('EPL').get_team_data('2022')


In [127]:
df = pd.DataFrame(t)#json_normalize(t)
set(df.reset_index().iloc[1].unique()[1:])

{'Arsenal',
 'Aston Villa',
 'Bournemouth',
 'Brentford',
 'Brighton',
 'Chelsea',
 'Crystal Palace',
 'Everton',
 'Fulham',
 'Leeds',
 'Leicester',
 'Liverpool',
 'Manchester City',
 'Manchester United',
 'Newcastle United',
 'Nottingham Forest',
 'Southampton',
 'Tottenham',
 'West Ham',
 'Wolverhampton Wanderers'}

In [374]:


from understatapi import UnderstatClient
understat = UnderstatClient()
years = range(2014,2024)
teams = set()
for year in years:
    t = understat.league('La_Liga').get_team_data(str(year))
    df = pd.DataFrame(t)
    ts = df.reset_index().iloc[1].unique()[1:]
    y = set(ts)
    teams.update(y)
#print(teams)
understat = UnderstatClient()
df = pd.DataFrame()
missing = {}
print(teams)
for team in teams:
    t = understat.team(team=team)
    print(team)
    for year in years:
        print(year)
        try:
            y = pd.json_normalize(t.get_match_data(str(year)))
            df = pd.concat([df, y])
        except:
            missing[year] = missing.get(year, []) + [team]
        
df


{'Rayo Vallecano', 'Espanyol', 'Athletic Club', 'Real Valladolid', 'Atletico Madrid', 'Eibar', 'Villarreal', 'Levante', 'Almeria', 'Alaves', 'Real Betis', 'Sevilla', 'Deportivo La Coruna', 'Cadiz', 'Leganes', 'Getafe', 'Osasuna', 'SD Huesca', 'Valencia', 'Las Palmas', 'Barcelona', 'Malaga', 'Granada', 'Cordoba', 'Real Madrid', 'Sporting Gijon', 'Mallorca', 'Elche', 'Celta Vigo', 'Real Sociedad', 'Girona'}
Rayo Vallecano
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
Espanyol
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
Athletic Club
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
Real Valladolid
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
Atletico Madrid
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
Eibar
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
Villarreal
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
Levante
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
Almeria
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
Alaves
2014
2015
2016
2017
2018
2019
2

,id,isResult,side,datetime,result,h.id,h.title,h.short_title,a.id,a.title,a.short_title,goals.h,goals.a,xG.h,xG.a,forecast.w,forecast.d,forecast.l
0,5835,True,h,2014-08-25 21:00:00,d,145,Rayo Vallecano,RVL,143,Atletico Madrid,ATL,0,0,0.357337,0.901586,0.141433,0.383080,0.475487
1,5843,True,a,2014-08-31 20:00:00,d,139,Deportivo La Coruna,DEP,145,Rayo Vallecano,RVL,2,2,1.89944,2.23586,0.335892,0.200879,0.463228
2,5791,True,h,2014-09-14 11:00:00,l,145,Rayo Vallecano,RVL,207,Elche,ELC,2,3,1.42044,0.570631,0.579321,0.271782,0.148897
3,5802,True,a,2014-09-21 16:00:00,l,154,Villarreal,VIL,145,Rayo Vallecano,RVL,4,2,4.38395,1.40506,0.854352,0.080726,0.064867
4,5809,True,h,2014-09-24 19:00:00,w,145,Rayo Vallecano,RVL,147,Athletic Club,ATH,2,1,3.17965,1.06179,0.786358,0.123061,0.090580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33,23019,False,h,2024-05-05 15:00:00,NaN,223,Girona,GIR,148,Barcelona,BAR,None,None,None,None,NaN,NaN,NaN
34,23030,False,a,2024-05-12 15:00:00,NaN,158,Alaves,ALA,223,Girona,GIR,None,None,None,None,NaN,NaN,NaN
35,23036,False,h,2024-05-15 15:00:00,NaN,223,Girona,GIR,154,Villarreal,VIL,None,None,None,None,NaN,NaN,NaN
36,23053,False,a,2024-05-19 15:00:00,NaN,146,Valencia,VAL,223,Girona,GIR,None,None,None,None,NaN,NaN,NaN


In [378]:
missing

{}

In [379]:
#t = understat.team(team='Arsenal')
#df = pd.json_normalize(t.get_match_data(str(2022)))
league = 'LaLiga'
df.to_csv(f'cache/{league}/understat.csv')

In [387]:
df1 = pd.read_csv(f'cache/{league}/understat.csv')
df1 = df1.sort_values('datetime')
df1 = df1.rename(columns={'h.title':'HomeTeam', 'a.title':'AwayTeam', 'datetime': 'Date',
'xG.h': 'HomeTeamxG', 'xG.a': 'AwayTeamxG'})

df1['Date'] = pd.to_datetime(df1['Date']).dt.strftime('%Y-%m-%d')
df1

,Unnamed: 0,id,isResult,side,Date,result,h.id,HomeTeam,h.short_title,a.id,AwayTeam,a.short_title,goals.h,goals.a,HomeTeamxG,AwayTeamxG,forecast.w,forecast.d,forecast.l
7980,0,5826,True,h,2014-08-23,w,137,Malaga,MAL,147,Athletic Club,ATH,1.0,0.0,1.321070,1.141510,0.407462,0.271098,0.321440
760,0,5826,True,a,2014-08-23,l,137,Malaga,MAL,147,Athletic Club,ATH,1.0,0.0,1.321070,1.141510,0.407462,0.271098,0.321440
6840,0,5827,True,a,2014-08-23,d,138,Sevilla,SEV,146,Valencia,VAL,1.0,1.0,1.171970,1.749030,0.255453,0.235605,0.508941
4560,0,5828,True,a,2014-08-23,l,155,Granada,GRA,139,Deportivo La Coruna,DEP,2.0,1.0,0.548351,0.379371,0.317067,0.482098,0.200835
4180,0,5827,True,h,2014-08-23,d,138,Sevilla,SEV,146,Valencia,VAL,1.0,1.0,1.171970,1.749030,0.255453,0.235605,0.508941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2659,37,23059,False,a,2024-05-26,NaN,157,Osasuna,OSA,154,Villarreal,VIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8511,37,23058,False,a,2024-05-26,NaN,223,Girona,GIR,155,Granada,GRA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8549,37,23058,False,a,2024-05-26,NaN,223,Girona,GIR,155,Granada,GRA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8739,37,23058,False,a,2024-05-26,NaN,223,Girona,GIR,155,Granada,GRA,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [388]:
#df = pd.read_csv('understat.csv')
df1 = df1.rename(columns={'xG.h': 'HomeTeamxG', 'xG.a': 'AwayTeamxG', 'result': 'Result'}
)
df1['Result'] = df1['Result'].str.upper()
df1 = df1.drop_duplicates(['id'])#['Date', 'HomeTeam', 'AwayTeam'])
df1

,Unnamed: 0,id,isResult,side,Date,Result,h.id,HomeTeam,h.short_title,a.id,AwayTeam,a.short_title,goals.h,goals.a,HomeTeamxG,AwayTeamxG,forecast.w,forecast.d,forecast.l
7980,0,5826,True,h,2014-08-23,W,137,Malaga,MAL,147,Athletic Club,ATH,1.0,0.0,1.321070,1.141510,0.407462,0.271098,0.321440
6840,0,5827,True,a,2014-08-23,D,138,Sevilla,SEV,146,Valencia,VAL,1.0,1.0,1.171970,1.749030,0.255453,0.235605,0.508941
4560,0,5828,True,a,2014-08-23,L,155,Granada,GRA,139,Deportivo La Coruna,DEP,2.0,1.0,0.548351,0.379371,0.317067,0.482098,0.200835
380,0,5829,True,a,2014-08-23,D,208,Almeria,ALM,141,Espanyol,ESP,1.0,1.0,0.978788,0.399306,0.491630,0.360626,0.147744
1900,0,5830,True,h,2014-08-24,W,156,Eibar,EIB,140,Real Sociedad,SOC,1.0,0.0,0.465398,0.975523,0.171885,0.357008,0.471107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4179,37,23063,False,a,2024-05-26,NaN,150,Real Madrid,RMA,153,Real Betis,BET,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1139,37,23060,False,a,2024-05-26,NaN,145,Rayo Vallecano,RVL,147,Athletic Club,ATH,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11665,37,23058,False,h,2024-05-26,NaN,223,Girona,GIR,155,Granada,GRA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11399,37,23061,False,h,2024-05-26,NaN,140,Real Sociedad,SOC,143,Atletico Madrid,ATL,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [389]:
class MissingDict(dict):
    __missing__ = lambda self, key: key
map_values = {
    'Middlesboro': 'Middlesbrough',
    "Nott'm Forest": 'Forest',
    'Newcastle Utd': 'Newcastle',
    'Luton Town': 'Luton',
    'Manchester City': 'Man City',
    'Manchester Utd': 'Man United',
    'Sheffield Utd': 'Sheffield United',
    "Nott'ham Forest": 'Forest',
    "Nottingham Forest": 'Forest',
    'Manchester United': 'Man United',
    'Wolverhampton Wanderers': 'Wolves',
    'Leicester City': 'Leicester',
    'Newcastle United': 'Newcastle',
    'Queens Park Rangers': 'QPR',
    'West Bromwich Albion': 'West Brom',
    'Vallecano': 'Rayo Vallecano', 
    'Ath Bilbao': 'Bilbao', 
    'Ath Madrid': 'Atletico', 
    'Espanol': 'Espanyol', 
    'Sp Gijon': 'Gijon', 
    'Celta Vigo': 'Celta', 
    'Atlético Madrid': 'Atletico', 
    'Athletic Club': 'Bilbao',
    'Real Sociedad': 'Sociedad',
    'Almería': 'Almeria',
    'Cádiz': 'Cadiz',
    'Alavés': 'Alaves',
    'Hellas Verona': 'Verona',
    'Atletico Madrid': 'Atletico', 
    'Sporting Gijon': 'Gijon',
    'Real Betis': 'Betis',
    'Real Valladolid': 'Valladolid'

}
mapping = MissingDict(**map_values)

def rename_cols(df):
    c1 = df.filter(like='HomeTeam').columns

    c2 = c1.str.replace('HomeTeam', 'Team')
    df = df.rename(columns={**dict(zip(c1, c2)), **dict(zip(c2, c1))})
    c1 = df.filter(like='AwayTeam').columns

    c2 = c1.str.replace('AwayTeam', 'Opponent')

    df = df.rename(columns={**dict(zip(c1, c2)), **dict(zip(c2, c1))})
    df['Team'] = df['Team'].map(mapping)
    df['Opponent'] = df['Opponent'].map(mapping)

    df['Venue'] = 'Home'
    return df
df2 = home_away(df1)#.drop_duplicates()
df2[df2['Date'] == '2023-09-24']


,Unnamed: 0,id,isResult,side,Date,Result,h.id,Team,h.short_title,a.id,Opponent,a.short_title,goals.h,goals.a,TeamxG,OpponentxG,forecast.w,forecast.d,forecast.l,Venue,GameID
6950,5,22743,True,h,2023-09-24,W,140,Sociedad,SOC,142,Getafe,GET,4.0,3.0,2.133840,1.593980,0.502482,0.208080,0.289438,Home,3475
6951,5,22743,True,h,2023-09-24,W,140,Getafe,SOC,142,Sociedad,GET,4.0,3.0,1.593980,2.133840,0.502482,0.208080,0.289438,Away,3475
6952,5,22741,True,h,2023-09-24,D,145,Villarreal,RVL,154,Rayo Vallecano,VIL,1.0,1.0,1.036500,1.034500,0.348443,0.302081,0.349476,Away,3476
6953,5,22741,True,h,2023-09-24,D,145,Rayo Vallecano,RVL,154,Villarreal,VIL,1.0,1.0,1.034500,1.036500,0.348443,0.302081,0.349476,Home,3476
6954,5,22738,True,a,2023-09-24,D,153,Real Betis,BET,261,Cadiz,CAD,1.0,1.0,2.707420,0.570946,0.822301,0.122730,0.054969,Home,3477
6955,5,22738,True,a,2023-09-24,D,153,Cadiz,BET,261,Real Betis,CAD,1.0,1.0,0.570946,2.707420,0.822301,0.122730,0.054969,Away,3477
6956,5,22744,True,h,2023-09-24,W,144,Granada,LPL,155,Las Palmas,GRA,1.0,0.0,0.037865,0.701293,0.491048,0.490280,0.018672,Away,3478
6957,5,22744,True,h,2023-09-24,W,144,Las Palmas,LPL,155,Granada,GRA,1.0,0.0,0.701293,0.037865,0.491048,0.490280,0.018672,Home,3478
6958,5,22736,True,h,2023-09-24,W,143,Real Madrid,ATL,150,Atletico Madrid,RMA,3.0,1.0,0.847915,1.795780,0.598186,0.228245,0.173569,Away,3479
6959,5,22736,True,h,2023-09-24,W,143,Atletico Madrid,ATL,150,Real Madrid,RMA,3.0,1.0,1.795780,0.847915,0.598186,0.228245,0.173569,Home,3479


In [373]:
df2.to_csv(f'cache/{league}/xG.csv')

In [385]:
elo = pd.read_csv(f'cache/{league}/elo.csv')

In [390]:
li1 = df2['Team'].unique()
s = set(elo['Team'].unique())

print([x for x in li1 if x not in s])

['Deportivo La Coruna', 'Atletico Madrid', 'Sporting Gijon', 'Real Betis', 'Real Valladolid', 'SD Huesca']


In [342]:
df[df['Date'] == '2023-09-24']

,Unnamed: 0,id,isResult,side,Date,Result,h.id,Team,h.short_title,a.id,Opponent,a.short_title,goals.h,goals.a,TeamxG,OpponentxG,forecast.w,forecast.d,forecast.l,Venue,GameID
6932,5,21951,True,h,2023-09-24,W,220,Brighton,BRI,73,Bournemouth,BOU,3.0,1.0,2.145330,1.589840,0.505717,0.207456,0.286827,Home,3466
6933,5,21951,True,h,2023-09-24,W,220,Bournemouth,BRI,73,Brighton,BOU,3.0,1.0,1.589840,2.145330,0.505717,0.207456,0.286827,Away,3466
6934,5,21947,True,a,2023-09-24,W,80,Aston Villa,CHE,71,Chelsea,AVL,0.0,1.0,1.134790,1.197160,0.374712,0.281202,0.344086,Away,3467
6935,5,21947,True,a,2023-09-24,W,80,Chelsea,CHE,71,Aston Villa,AVL,0.0,1.0,1.197160,1.134790,0.374712,0.281202,0.344086,Home,3467
6936,5,21954,True,h,2023-09-24,L,238,Newcastle,SHE,86,Sheffield United,NEW,0.0,8.0,5.108200,0.579392,0.008346,0.023782,0.967590,Away,3468
6937,5,21954,True,h,2023-09-24,L,238,Sheffield United,SHE,86,Newcastle,NEW,0.0,8.0,0.579392,5.108200,0.008346,0.023782,0.967590,Home,3468


In [344]:

#elo = elo[elo['Date'] >= '2014-08']
elo
new1 = elo.merge(df[['Date','Team', 'TeamxG', 'OpponentxG', 'Opponent']], on=['Date','Team', 'Opponent'], how='left').drop_duplicates()
new1.isna().sum()

new1.to_csv('xG.csv')
#new1.merge(df[['Date','Team', 'TeamxG', 'OpponentxG', 'Opponent']], on=['Date', 'Opponent'], how='left').drop_duplicates()

In [352]:
sched = pd.read_csv(f'cache/PremierLeague/schedule.csv')
sched = rename_cols(sched)
sched
sched.merge(df[['Date', 'Team', 'TeamxG', 'OpponentxG']], on=['Date', 'Team'])

,Unnamed: 0,week,day,Date,time,Team,home_xg,score,away_xg,Opponent,attendance,venue,Referee,match_report,notes,game_id,BbAv>2.5,BbAv<2.5,BbAvH,BbAvD,BbAvA,Venue
0,0,1,Fri,2023-08-11,20:00,Burnley,0.3,0–3,1.9,Man City,21572.0,Turf Moor,Craig Pawson,/en/matches/3a6836b4/Burnley-Manchester-City-A...,NaN,3a6836b4,NaN,NaN,NaN,NaN,NaN,Home
1,1,1,Sat,2023-08-12,12:30,Arsenal,0.8,2–1,1.2,Forest,59984.0,Emirates Stadium,Michael Oliver,/en/matches/26a7f90c/Arsenal-Nottingham-Forest...,NaN,26a7f90c,NaN,NaN,NaN,NaN,NaN,Home
2,2,1,Sat,2023-08-12,15:00,Bournemouth,1.3,1–1,1.1,West Ham,11245.0,Vitality Stadium,Peter Bankes,/en/matches/d6bbf293/Bournemouth-West-Ham-Unit...,NaN,d6bbf293,NaN,NaN,NaN,NaN,NaN,Home
3,3,1,Sat,2023-08-12,15:00,Brighton,4.0,4–1,1.5,Luton,31872.0,The American Express Community Stadium,David Coote,/en/matches/56a137f7/Brighton-and-Hove-Albion-...,NaN,56a137f7,NaN,NaN,NaN,NaN,NaN,Home
4,4,1,Sat,2023-08-12,15:00,Everton,2.7,0–1,1.5,Fulham,39940.0,Goodison Park,Stuart Attwell,/en/matches/15addfc7/Everton-Fulham-August-12-...,NaN,15addfc7,NaN,NaN,NaN,NaN,NaN,Home
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,375,38,Sun,2024-05-19,NaN,Crystal Palace,NaN,NaN,NaN,Aston Villa,NaN,Selhurst Park,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Home
376,376,38,Sun,2024-05-19,NaN,Liverpool,NaN,NaN,NaN,Wolves,NaN,Anfield,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Home
377,377,38,Sun,2024-05-19,NaN,Luton,NaN,NaN,NaN,Fulham,NaN,Kenilworth Road Stadium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Home
378,378,38,Sun,2024-05-19,NaN,Man City,NaN,NaN,NaN,West Ham,NaN,Etihad Stadium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Home


In [28]:
fbref = sd.FBref(leagues='ENG-Premier League', seasons=2019)
fbref.read_team_match_stats()

[09/28/23 12:33:52] INFO     Saving cached data to /Users/Gautham/soccerdata/data/FBref               ]8;id=399759;file:///Users/Gautham/miniconda3/envs/dl_hw0/lib/python3.8/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=390898;file:///Users/Gautham/miniconda3/envs/dl_hw0/lib/python3.8/site-packages/soccerdata/_common.py#89\89]8;;\

date  \
league             season team    game                                          
ENG-Premier League 1920   Arsenal 2019-08-11 Newcastle Utd-Arsenal 2019-08-11   
                                  2019-08-17 Arsenal-Burnley       2019-08-17   
                                  2019-08-24 Liverpool-Arsenal     2019-08-24   
                                  2019-09-01 Arsenal-Tottenham     2019-09-01   
                                  2019-09-15 Watford-Arsenal       2019-09-15   
...                                                                       ...   
                          Wolves  2020-07-08 Sheffield Utd-Wolves  2020-07-08   
                                  2020-07-12 Wolves-Everton        2020-07-12   
                                  2020-07-15 Burnley-Wolves        2020-07-15   
                                  2020-07-20 Wolves-Crystal Palace 2020-07-20   
                                  2020-07-26 Chelsea-Wolves        2020-07-26   

                                                                        time  \
league             season team    game                                         
ENG-Premier League 1920   Arsenal 2019-08-11 Newcastle Utd-Arsenal  14:00:00   
                                  2019-08-17 Arsenal-Burnley        12:30:00   
                                  2019-08-24 Liverpool-Arsenal      17:30:00   
                                  2019-09-01 Arsenal-Tottenham      16:30:00   
                                  2019-09-15 Watford-Arsenal        16:30:00   
...                                                                      ...   
                          Wolves  2020-07-08 Sheffield Utd-Wolves   18:00:00   
                                  2020-07-12 Wolves-Everton         12:00:00   
                                  2020-07-15 Burnley-Wolves         18:00:00   
                                  2020-07-20 Wolves-Crystal Palace  20:15:00   
                                  2020-07-26 Chelsea-Wolves         16:00:00   

                                                                           round  \
league             season team    game                                             
ENG-Premier League 1920   Arsenal 2019-08-11 Newcastle Utd-Arsenal   Matchweek 1   
                                  2019-08-17 Arsenal-Burnley         Matchweek 2   
                                  2019-08-24 Liverpool-Arsenal       Matchweek 3   
                                  2019-09-01 Arsenal-Tottenham       Matchweek 4   
                                  2019-09-15 Watford-Arsenal         Matchweek 5   
...                                                                          ...   
                          Wolves  2020-07-08 Sheffield Utd-Wolves   Matchweek 34   
                                  2020-07-12 Wolves-Everton         Matchweek 35   
                                  2020-07-15 Burnley-Wolves         Matchweek 36   
                                  2020-07-20 Wolves-Crystal Palace  Matchweek 37   
                                  2020-07-26 Chelsea-Wolves         Matchweek 38   

                                                                    day venue  \
league             season team    game                                          
ENG-Premier League 1920   Arsenal 2019-08-11 Newcastle Utd-Arsenal  Sun  Away   
                                  2019-08-17 Arsenal-Burnley        Sat  Home   
                                  2019-08-24 Liverpool-Arsenal      Sat  Away   
                                  2019-09-01 Arsenal-Tottenham      Sun  Home   
                                  2019-09-15 Watford-Arsenal        Sun  Away   
...                                                                 ...   ...   
                          Wolves  2020-07-08 Sheffield Utd-Wolves   Wed  Away   
                                  2020-07-12 Wolves-Everton         Sun  Home   
                                  2020-07-15 Burnley-Wolves         Wed  Aw

In [19]:
#! pip install pandasql
import pandasql as ps
processed = pd.read_csv('cache/PremierLeague/processed.csv')
clubelo = sd.ClubElo('ENG-Premier League')#self.dic[self.league])
missing = []
# create elo df
elo_df = pd.DataFrame()
#print(full_df['Team'].unique())
for club in processed['Team'].unique():
    print('hi', club)
    try:
        elo = clubelo.read_team_history(club)
        elo_df = pd.concat([elo_df,elo])
    except:
        missing.append(club)
    #print(elo_df)
print(missing)
elo_df = elo_df.reset_index()
elo_df = elo_df[elo_df['from'] > '2000'] 



[09/27/23 13:11:45] INFO     Saving cached data to /Users/Gautham/soccerdata/data/ClubElo             ]8;id=79739;file:///Users/Gautham/miniconda3/envs/dl_hw0/lib/python3.8/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=517298;file:///Users/Gautham/miniconda3/envs/dl_hw0/lib/python3.8/site-packages/soccerdata/_common.py#89\89]8;;\

hi Charlton
hi Man City
hi Chelsea
hi West Ham
hi Coventry
hi Middlesbrough
hi Southampton
hi Derby
hi Leeds
hi Everton
hi Leicester
hi Aston Villa
hi Bradford
hi Liverpool
hi Arsenal
hi Sunderland
hi Ipswich
hi Tottenham
hi Newcastle
hi Man United
hi Bolton
hi Blackburn
hi Fulham
hi West Brom
hi Birmingham
hi Portsmouth
hi Wolves
hi Norwich
hi Crystal Palace
hi Wigan
hi Watford
hi Reading
hi Sheffield United
hi Hull
hi Stoke
hi Burnley
hi Blackpool
hi QPR
hi Swansea
hi Cardiff
hi Bournemouth
hi Brighton
hi Huddersfield
hi Brentford
hi Forest
hi Luton
[]


In [26]:
out = pd.merge_asof(processed, elo_df[['team', 'elo', 'from', 'to']], left_on=['Team', 'Date'], right_on=['team', 'from'], allow_exact_matches=True) 
#out = out.query('Date.between(`from`, `to`)')
#out = out.merge(elo_df[['team', 'elo', 'from', 'to']], how='left', left_on=['Opponent'], right_on=['team']) 
#out = out.query('Date.between(`from_y`, `to_y`)')
out

MergeError: can only asof on a key for left

In [121]:
sqlcode = '''
SELECT * FROM processed as a join elo_df as b on a.Team=b.team where a.Date >= b.from and a.Date <= b.to
'''
sqlcode = 'SELECT * from processed join elo_df'

newdf = ps.sqldf(sqlcode,locals())
newdf

In [119]:
newdf

,Unnamed: 0,GameID,Date,Team,Opponent,TotalGoals,TeamGoals,OpponentGoals,Result,Referee,TeamShots,OpponentShots,TeamShotsonTarget,OpponentShotsonTarget,TeamFouls,OpponentFouls,TeamCorners,OpponentCorners,TeamYellowCards,OpponentYellowCards,TeamRedCards,OpponentRedCards,O2.5,U2.5,TeamWinOdds,DrawOdds,OpponentWinOdds,Venue
0,0,0,2000-08-19,Charlton,Man City,4,4,0,W,Rob Harris,17,8,14,4,13,12,6,6,1,2,0,0,NaN,NaN,NaN,NaN,NaN,Home
1,1,0,2000-08-19,Man City,Charlton,4,0,4,L,Rob Harris,8,17,4,14,12,13,6,6,2,1,0,0,NaN,NaN,NaN,NaN,NaN,Away
2,2,1,2000-08-19,Chelsea,West Ham,6,4,2,W,Graham Barber,17,12,10,5,19,14,7,7,1,2,0,0,NaN,NaN,NaN,NaN,NaN,Home
3,3,1,2000-08-19,West Ham,Chelsea,6,2,4,L,Graham Barber,12,17,5,10,14,19,7,7,2,1,0,0,NaN,NaN,NaN,NaN,NaN,Away
4,4,2,2000-08-19,Coventry,Middlesbrough,4,1,3,L,Barry Knight,6,16,3,9,15,21,8,4,5,3,1,0,NaN,NaN,NaN,NaN,NaN,Home
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16893,16893,8446,2023-09-17,Arsenal,Everton,1,1,0,W,S Hooper,13,8,4,1,10,12,11,1,1,1,0,0,1.67,2.23,1.50,4.50,6.00,Away
16894,16894,8447,2023-09-17,Bournemouth,Chelsea,0,0,0,D,D Coote,13,14,4,6,15,20,1,7,1,5,0,0,1.67,2.19,4.33,3.75,1.80,Home
16895,16895,8447,2023-09-17,Chelsea,Bournemouth,0,0,0,D,D Coote,14,13,6,4,20,15,7,1,5,1,0,0,1.67,2.19,1.80,3.75,4.33,Away
16896,16896,8448,2023-09-18,Forest,Burnley,2,1,1,D,R Jones,14,10,4,3,16,10,5,4,2,3,0,1,1.92,1.87,2.20,3.50,3.25,Home
